In [66]:
# Facebook签到位置预测

In [67]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [68]:
# 导入数据
data = pd.read_csv("../data/FBlocation/train.csv")
data

,row_id,x,y,accuracy,time,place_id
0,0,0.7941,9.0809,54,470702,8523065625
1,1,5.9567,4.7968,13,186555,1757726713
2,2,8.3078,7.0407,74,322648,1137537235
3,3,7.3665,2.5165,65,704587,6567393236
4,4,4.0961,1.1307,31,472130,7440663949
...,...,...,...,...,...,...
29118016,29118016,6.5133,1.1435,67,399740,8671361106
29118017,29118017,5.9186,4.4134,67,125480,9077887898
29118018,29118018,2.9993,6.3680,67,737758,2838334300
29118019,29118019,4.0637,8.0061,70,764975,1007355847


In [69]:
# 数据逻辑筛选（缩小数据集）
data = data.query("x > 2 & x < 2.5 & y > 1 & y < 1.5")
data

,row_id,x,y,accuracy,time,place_id
112,112,2.2360,1.3655,66,623174,7663031065
180,180,2.2003,1.2541,65,610195,2358558474
367,367,2.4108,1.3213,74,579667,6644108708
874,874,2.0822,1.1973,320,143566,3229876087
1022,1022,2.0160,1.1659,65,207993,3244363975
...,...,...,...,...,...,...
29115112,29115112,2.1889,1.2914,168,721885,4606837364
29115204,29115204,2.1193,1.4692,58,563389,2074133146
29115338,29115338,2.0007,1.4852,25,765986,6691588909
29115464,29115464,2.4132,1.4237,61,151918,7396159924


In [70]:
# 时间特征处理
time_value = pd.to_datetime(data["time"], unit="s")
time_value

112        1970-01-08 05:06:14
180        1970-01-08 01:29:55
367        1970-01-07 17:01:07
874        1970-01-02 15:52:46
1022       1970-01-03 09:46:33
                   ...        
29115112   1970-01-09 08:31:25
29115204   1970-01-07 12:29:49
29115338   1970-01-09 20:46:26
29115464   1970-01-02 18:11:58
29117493   1970-01-01 22:06:09
Name: time, Length: 83197, dtype: datetime64[ns]

In [71]:
date = pd.DatetimeIndex(time_value)
date

DatetimeIndex(['1970-01-08 05:06:14', '1970-01-08 01:29:55',
               '1970-01-07 17:01:07', '1970-01-02 15:52:46',
               '1970-01-03 09:46:33', '1970-01-06 19:49:38',
               '1970-01-06 13:33:24', '1970-01-02 22:49:55',
               '1970-01-04 14:30:10', '1970-01-07 16:57:44',
               ...
               '1970-01-02 09:24:50', '1970-01-01 10:29:34',
               '1970-01-09 11:38:46', '1970-01-02 03:42:14',
               '1970-01-04 22:02:44', '1970-01-09 08:31:25',
               '1970-01-07 12:29:49', '1970-01-09 20:46:26',
               '1970-01-02 18:11:58', '1970-01-01 22:06:09'],
              dtype='datetime64[ns]', name='time', length=83197, freq=None)

In [72]:
# 新增特征
data["day"] = date.day
data["weekday"] = date.weekday
data["hour"] = date.hour
data

C:\Users\Administrator\AppData\Local\Temp\ipykernel_11220\1070596042.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["day"] = date.day
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11220\1070596042.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["weekday"] = date.weekday
C:\Users\Administrator\AppData\Local\Temp\ipykernel_11220\1070596042.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
180,180,2.2003,1.2541,65,610195,2358558474,8,3,1
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9
...,...,...,...,...,...,...,...,...,...
29115112,29115112,2.1889,1.2914,168,721885,4606837364,9,4,8
29115204,29115204,2.1193,1.4692,58,563389,2074133146,7,2,12
29115338,29115338,2.0007,1.4852,25,765986,6691588909,9,4,20
29115464,29115464,2.4132,1.4237,61,151918,7396159924,2,4,18


In [73]:
# 删除 place_id 次数少于3次的数据
place_count = data.groupby("place_id").count()   # 分组与聚合
place_count

,row_id,x,y,accuracy,time,day,weekday,hour
place_id,,,,,,,,
1012165853,1,1,1,1,1,1,1,1
1013991737,3,3,3,3,3,3,3,3
1014605271,28,28,28,28,28,28,28,28
1015645743,4,4,4,4,4,4,4,4
1017236154,31,31,31,31,31,31,31,31
...,...,...,...,...,...,...,...,...
9988815170,1,1,1,1,1,1,1,1
9992113332,1,1,1,1,1,1,1,1
9993074125,2,2,2,2,2,2,2,2


In [74]:
place_count = place_count["row_id"]
place_count

place_id
1012165853     1
1013991737     3
1014605271    28
1015645743     4
1017236154    31
              ..
9988815170     1
9992113332     1
9993074125     2
9994257798    25
9996671132    18
Name: row_id, Length: 2514, dtype: int64

In [75]:
place_count[place_count > 3]

place_id
1014605271     28
1015645743      4
1017236154     31
1024951487      5
1028119817      4
             ... 
9936666116    140
9954155328      8
9980625005     16
9994257798     25
9996671132     18
Name: row_id, Length: 950, dtype: int64

In [76]:
data["place_id"].isin(place_count[place_count > 3].index.values)

112          True
180         False
367          True
874          True
1022         True
            ...  
29115112     True
29115204     True
29115338     True
29115464     True
29117493     True
Name: place_id, Length: 83197, dtype: bool

In [77]:
data_final = data[data["place_id"].isin(place_count[place_count > 3].index.values)]
data_final

,row_id,x,y,accuracy,time,place_id,day,weekday,hour
112,112,2.2360,1.3655,66,623174,7663031065,8,3,5
367,367,2.4108,1.3213,74,579667,6644108708,7,2,17
874,874,2.0822,1.1973,320,143566,3229876087,2,4,15
1022,1022,2.0160,1.1659,65,207993,3244363975,3,5,9
1045,1045,2.3859,1.1660,498,503378,6438240873,6,1,19
...,...,...,...,...,...,...,...,...,...
29115112,29115112,2.1889,1.2914,168,721885,4606837364,9,4,8
29115204,29115204,2.1193,1.4692,58,563389,2074133146,7,2,12
29115338,29115338,2.0007,1.4852,25,765986,6691588909,9,4,20
29115464,29115464,2.4132,1.4237,61,151918,7396159924,2,4,18


In [78]:
# 特征值
x = data_final[["x", "y", "accuracy", "day", "weekday", "hour"]]
x

,x,y,accuracy,day,weekday,hour
112,2.2360,1.3655,66,8,3,5
367,2.4108,1.3213,74,7,2,17
874,2.0822,1.1973,320,2,4,15
1022,2.0160,1.1659,65,3,5,9
1045,2.3859,1.1660,498,6,1,19
...,...,...,...,...,...,...
29115112,2.1889,1.2914,168,9,4,8
29115204,2.1193,1.4692,58,7,2,12
29115338,2.0007,1.4852,25,9,4,20
29115464,2.4132,1.4237,61,2,4,18


In [79]:
# 目标值
y = data_final["place_id"]
y

112         7663031065
367         6644108708
874         3229876087
1022        3244363975
1045        6438240873
               ...    
29115112    4606837364
29115204    2074133146
29115338    6691588909
29115464    7396159924
29117493    1168869217
Name: place_id, Length: 80910, dtype: int64

In [80]:
# 数据集划分
x_train, x_test, y_train, y_test = train_test_split(x, y)
x_train

,x,y,accuracy,day,weekday,hour
28158054,2.4145,1.0966,171,3,5,1
5663146,2.3018,1.1450,49,1,3,15
8496201,2.2522,1.4433,64,7,2,23
3877155,2.0046,1.3443,962,9,4,16
18924963,2.0758,1.0233,9,2,4,6
...,...,...,...,...,...,...
11346278,2.4412,1.4127,45,6,1,22
26439970,2.3259,1.0900,75,5,0,6
27747232,2.0260,1.1801,5,1,3,21
7297251,2.0452,1.4542,10,5,0,8


In [81]:
y_train

28158054    3411026024
5663146     3109132734
8496201     7920779710
3877155     5992403068
18924963    5600661085
               ...    
11346278    5263230464
26439970    5541564636
27747232    9764078387
7297251     5232128865
19273755    8790355618
Name: place_id, Length: 60682, dtype: int64

In [82]:
x_test

,x,y,accuracy,day,weekday,hour
16081117,2.2535,1.2486,84,3,5,9
16586404,2.4295,1.1693,22,7,2,22
20161710,2.3461,1.4619,210,3,5,2
9301042,2.1196,1.2355,65,4,6,20
20788605,2.2358,1.2086,89,2,4,20
...,...,...,...,...,...,...
8876823,2.2211,1.0815,71,3,5,11
21775618,2.0963,1.3046,52,5,0,12
18890156,2.1322,1.1268,11,7,2,10
937959,2.2663,1.0357,19,8,3,17


In [83]:
y_test

16081117    5448001036
16586404    7577024623
20161710    3002774559
9301042     1380291167
20788605    7312165178
               ...    
8876823     8833193339
21775618    7605688218
18890156    4691678575
937959      7810150060
7588449     4033454985
Name: place_id, Length: 20228, dtype: int64

In [84]:
# 转换器-标准化
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

In [85]:
# 估计器
estimator = KNeighborsClassifier()
# 网格搜索交叉验证
param_dic = {"n_neighbors": [3, 5, 7, 9]}
estimator = GridSearchCV(estimator, param_grid=param_dic, cv=3)
# 模型训练
estimator.fit(x_train, y_train)



y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接对比真实值和预测值：\n", y_test == y_predict)

D:\Soft\Python\Python310\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(


y_predict:
 [6215057357 6396652784 1853388867 ... 4691678575 1465956877 9773056775]
直接对比真实值和预测值：
 16081117    False
16586404    False
20161710    False
9301042      True
20788605    False
            ...  
8876823     False
21775618    False
18890156     True
937959      False
7588449     False
Name: place_id, Length: 20228, dtype: bool


In [86]:
# 模型评估
# 方法1：直接对比真实值和预测值
y_predict = estimator.predict(x_test)
print("y_predict:\n", y_predict)
print("直接对比真实值和预测值：\n", y_test == y_predict)
# 方法2：计算准确率
score = estimator.score(x_test, y_test)
print("准确率为：\n", score)

y_predict:
 [6215057357 6396652784 1853388867 ... 4691678575 1465956877 9773056775]
直接对比真实值和预测值：
 16081117    False
16586404    False
20161710    False
9301042      True
20788605    False
            ...  
8876823     False
21775618    False
18890156     True
937959      False
7588449     False
Name: place_id, Length: 20228, dtype: bool
准确率为：
 0.3653351789598576


In [87]:
# 最佳参数
print("最佳参数：\n", estimator.best_params_)
# 最佳结果
print("最佳结果：\n", estimator.best_score_)
# 最佳估计器
print("最佳估计器:\n", estimator.best_estimator_)
# 交叉验证结果
print("交叉验证结果：\n", estimator.cv_results_)

最佳参数：
 {'n_neighbors': 7}
最佳结果：
 0.33141621825684325
最佳估计器:
 KNeighborsClassifier(n_neighbors=7)
交叉验证结果：
 {'mean_fit_time': array([0.06468121, 0.06434727, 0.06434751, 0.06568138]), 'std_fit_time': array([0.00047131, 0.00047204, 0.00047171, 0.00047131]), 'mean_score_time': array([0.50611448, 0.54312293, 0.57813112, 0.59180029]), 'std_score_time': array([0.00432198, 0.00216097, 0.00141439, 0.00818032]), 'param_n_neighbors': masked_array(data=[3, 5, 7, 9],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'n_neighbors': 9}], 'split0_test_score': array([0.32267154, 0.33127348, 0.33245996, 0.32539055]), 'split1_test_score': array([0.32278637, 0.32960894, 0.32871904, 0.32595046]), 'split2_test_score': array([0.32234142, 0.33213032, 0.33306966, 0.32852128]), 'mean_test_score': array([0.32259978, 0.33100425, 0.33141622, 0.32662076]), 'std_test_score': array([0.0001886 , 0.001